# Phenotype data preprocessing

This mini-protocol documents the shared post processing step and some utilities to handle molecular phenotype files including imputations.

#### Miniprotocol Timing
This represents the total duration for all miniprotocol phases. While module-specific timings are provided separately on their respective pages, they are also included in this overall estimate. 

Timing < X minutes

## Overview

This workflow is an application of the phenotype related workflows from the xQTL project pipeline.

1. `gene_annotation.ipynb` (step i): Adds genomic coordinate annotation to gene-level molecular phenotype files and converts them to `.bed` format
2. `phenotype_imputation.ipynb` (step ii): [FIXME: to be updated]
3. `phenotype_formatting.ipynb` (step iii): Splits each phenotype file by chromosome


## Steps

### i. Phenotype Annotation
This step serves as annote cooresponding `chr`, `start`, `end`, and `gene_id` to genes in the original phenotype matrix. 

In [ ]:
sos run pipeline/gene_annotation.ipynb annotate_coord_protein \
    --cwd output/phenotype \
    --phenoFile input/protocol_example.protein.csv \
    --annotation-gtf reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.ERCC.gtf \
    --phenotype-id-type gene_name \
    --sample-participant-lookup output/sample_meta/protocol_example.protein.sample_overlap.txt \
    --container containers/rna_quantification.sif

### ii. Mean Imputation

FIXME: mention that for eQTL it is fine to skip this.

FIXME: For this step we are still working out the best approach. This will be updated with the proper imputation command once we finalize on the simulation results (currently very much likely, using `flashier`).

### iii. Partition by Chromosome

In [ ]:
sos run pipeline/phenotype_formatting.ipynb phenotype_by_chrom \
    --cwd output/phenotype_by_chrom \
    --phenoFile output/phenotype/protocol_example.protein.bed.gz \
    --chrom `for i in {21..22}; do echo chr$i; done` \
    --container containers/bioinfo.sif

## Anticipated Results